In [5]:
import numpy as np
import gensim
import re

In [8]:
clean = ['warandpeace.txt', 'annakarenina.txt', 'idiot.txt', 'onegin.txt']
medium = ['choke.txt', 'fightclub.txt', 'lullaby.txt', 'masterandmargaret.txt', 'messenger.txt',
         'pulpfiction.txt', 'survivor.txt']
slang = ['mitiki.txt', 'rastamans.txt', 'chapaev_demo.txt']

In [50]:
names = [clean, medium, slang]
texts = []
fulls = []
sets = []

for group in names:
    docs = []
    full = ''
    for book in group:
        corpus = []
        with open(book) as file:
            for line in file:
                corpus.append(line)

        corpus = [el for el in corpus if el != '\n']
        corpus = [re.sub(r'[^\w\s]','',el)[:-1] for el in corpus]
        for sentence in corpus:
            full += ' ' + sentence
        docs += [el.lower().split() for el in corpus]
        
    words = []
    for sentence in docs:
        words += sentence
    sets.append(set(words))
    texts += docs
    fulls.append(full)

In [37]:
model = gensim.models.Word2Vec(texts, size=100, iter=70, workers=4, min_count=1)

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [54]:
vec = TfidfVectorizer()
tfidf = vec.fit_transform(fulls)

In [88]:
npidf = tfidf.toarray()

In [73]:
fnames = vec.get_feature_names()

In [101]:
fnames.index('забастовали')

30002

In [102]:
npidf[:,30002]

array([0.       , 0.       , 0.0009569])

In [96]:
for i, word in enumerate(fnames):
    idf = npidf[:, i]
    if (idf[1] > 100 * idf[0]) and (idf[2] > 100 * idf[1]):
        print(word, idf)

менты [0.00000000e+00 8.19675923e-05 2.11046352e-02]
партизан [0.00000000e+00 8.19675923e-05 1.96491431e-02]
суки [0.00000000e+00 8.19675923e-05 8.73295248e-03]
хуй [0.00000000e+00 8.19675923e-05 3.27485718e-02]
хули [0.00000000e+00 8.19675923e-05 1.01884446e-02]
хуя [0.00000000e+00 8.19675923e-05 1.23716827e-02]
чувак [0.00000000e+00 8.19675923e-05 2.40156193e-02]


In [130]:
test = 'чувак слышишь гром пушек над рекой чертовы менты задолбали ненавижу французы бородино'
test2 = 'фигня хрень смерть турист растаман'
test3 = 'я иду вижу мента он стреляет революция нахер все снесем к чертям собачьим'
test4 = 'король пришел забабахал авиакатастрофы смерть младенцы кровь тоска кишки россия'

In [120]:
def logic(string):
    for word in string.split():
        try:
            index = fnames.index(word)
            idf = npidf[:, index]
            if (idf[1] > 100 * idf[0]) or (idf[2] > 100 * idf[0]):
                best = '0'
                best_res = -100
                for gword in sets[0]:
                    score = model.wv.similarity(word, gword)
                    if score > best_res:
                        best_res = score
                        best = gword
                print(word, ' ---> ', best)
        except:
            pass

In [131]:
logic(test4)

авиакатастрофы  --->  старческий


In [110]:
model.wv.most_similar('высокомерно')

[('изможденном', 0.7472450733184814),
 ('восхитительна', 0.7293205261230469),
 ('кутузовскою', 0.7288143634796143),
 ('тревожная', 0.7285295724868774),
 ('таблиц', 0.7273244857788086),
 ('неумолимых', 0.7271543741226196),
 ('единственна', 0.7251404523849487),
 ('взывала', 0.7242549061775208),
 ('благословением', 0.723096489906311),
 ('очерстветь', 0.7229198813438416)]